# **Basic system setting**


In [ ]:
!nvidia-smi

In [ ]:
import os 
print(os.getcwd())

In [ ]:
!pip install autotorch thop torch-encoding

In [ ]:
!pip3 uninstall apex
!pip install git+https://www.github.com/nvidia/apex

In [ ]:
!pip install thop

In [ ]:
!pip3 install resume

In [ ]:
!pip install git+https://github.com/1adrianb/pytorch-estimate-flops && pytorch-estimate-flops

In [ ]:
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

In [ ]:
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
!update-alternatives: using /usr/local/bin/ninja to provide /usr/bin/ninja (ninja) in auto mode

In [ ]:
!/usr/bin/ninja --version

/bin/bash: /usr/bin/ninja: No such file or directory


In [ ]:
!pip install argparse

In [ ]:
!pip install arch

In [ ]:
!pip install datetime

In [ ]:
!pip install torchviz

In [ ]:
!pip show autotorch

# **Training Beginning**

In [ ]:
import argparse
import h5py
import os
import sys
import shutil
import time
import datetime
import numpy as np
from torchsummary import summary
from torchviz import make_dot


import torch
print(torch.__version__)
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data as Data
from torch.utils.data import DataLoader
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter

sys.path.append('../content/drive/My Drive/RegNet_Research')

from src.regnet import RegNetY
from src.config import TRAIN_IMAGE_SIZE
  

def transfer_dataset(train_E1_set, test_E1_set):
  # extract label from group
    train_E1_label = train_E1_set[:, 0].reshape(len(train_E1_set), 1)
    test_E1_label = test_E1_set[:, 0].reshape(len(test_E1_set), 1)

    # obtain items and labels from train_bangli_data
    tl = torch.LongTensor(train_E1_label)
    sl = torch.LongTensor(test_E1_label)

    # obtain train data
    train_data = []
    for i in range(len(train_E1_set)):
        td = train_E1_set[i][1:].reshape(16, 256)
        td = np.rot90(td, 1)
        train_data.append(td)
    train_data = np.array(train_data)
    training_data = torch.from_numpy(train_data)
    training_data = torch.unsqueeze(training_data, dim=1).type(torch.FloatTensor)
    training_data = torch.cat((training_data, training_data, training_data), 1)
    print(training_data.shape)

    # obtain test data
    test_data = []
    for i in range(len(test_E1_set)):
        sd = test_E1_set[i][1:].reshape(16, 256)
        sd = np.rot90(sd, 1)
        test_data.append(sd)
    test_data = np.array(test_data)
    testing_data = torch.from_numpy(test_data)
    testing_data = torch.unsqueeze(testing_data, dim=1).type(torch.FloatTensor)
    testing_data = torch.cat((testing_data, testing_data, testing_data), 1)
    print(testing_data.shape)

    training_label = torch.squeeze(tl)
    testing_label = torch.squeeze(sl)

    dataset = Data.TensorDataset(training_data, training_label)  # build dataset
    dataset2 = Data.TensorDataset(testing_data, testing_label)  # build dataset2
    return dataset, dataset2


def get_args():
    parser = argparse.ArgumentParser(
          description="Implementation of paradigm described in paper: Designing Network Design Spaces published by Facebook AI Research (FAIR)")

    parser.add_argument("-d", "--data_path", type=str, default="data", help="the root folder of dataset")
    parser.add_argument("-e", "--epochs", default=20, type=int, help="number of total epochs to run")
    parser.add_argument("-b", "--batch_size", default=128, type=int)
    parser.add_argument("-l", "--lr", default=0.1, type=float, help="initial learning rate")
    parser.add_argument("-m", "--momentum", default=0.9, type=float, help="momentum")
    parser.add_argument("-w", "--weight_decay", default=5e-4, type=float, help="weight decay")
    parser.add_argument("--log_path", type=str, default="tensorboard/signatrix_regnet_imagenet")
    parser.add_argument("--saved_path", type=str, default="trained_models")

    # These default parameters are for RegnetX 400MF
    parser.add_argument("--bottleneck_ratio", default=1, type=int)
    parser.add_argument("--group_width", default=160, type=int)
    parser.add_argument("--initial_width", default=231, type=int)
    parser.add_argument("--slope", default=47.01397787532618, type=float)
    parser.add_argument("--quantized_param", default=3.1614385630517594, type=float)
    parser.add_argument("--network_depth", default=27, type=int)
    parser.add_argument("--stride", default=2, type=int)
    parser.add_argument("--se_ratio", default=4, type=int)
    parser.add_argument('-f')
    args = parser.parse_args()
    return args


def adjust_learning_rate(optimizer, epoch, lr):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = lr * (0.1 ** (epoch // 10))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    print('lr', lr)


def main(opt):
    num_gpus = 1
    if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        torch.cuda.manual_seed(123)
    else:
        torch.manual_seed(123)

    Database = h5py.File('/content/drive/My Drive/ISRMyo-I.h5', 'r')

    train_set = Database['train_set']
    test_set = Database['test_set']

    # obtain training data & testing data
    train_set_E1 = train_set['train_bangli']
    train_set_E1 = np.array(train_set_E1)
    test_set_E1 = test_set['test_bangli']
    test_set_E1 = np.array(test_set_E1)
    print('training_data', train_set_E1.shape)
    print('testing_data', test_set_E1.shape)

    train_E1_set = train_set_E1
    test_E1_set = test_set_E1

    training_set, test_set = transfer_dataset(train_set_E1, test_set_E1)
    training_generator = DataLoader(training_set, 
                                    batch_size=128,
                                    shuffle=True,
                                    drop_last=True,
                                    num_workers=12)
    test_generator = DataLoader(test_set, 
                                batch_size=128,
                                shuffle=False,
                                drop_last=False,
                                num_workers=12)

    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)

    if not os.path.isdir(opt.saved_path):
        os.makedirs(opt.saved_path)

    writer = SummaryWriter(opt.log_path)
    model = RegNetY(opt.initial_width, opt.slope, opt.quantized_param, opt.network_depth, opt.bottleneck_ratio,
                    opt.group_width, opt.stride, opt.se_ratio)
    dummy_input = torch.randn((1, 3, TRAIN_IMAGE_SIZE[0], TRAIN_IMAGE_SIZE[1]))

    writer.add_graph(model, dummy_input)
    summary(model, (3, TRAIN_IMAGE_SIZE[0], TRAIN_IMAGE_SIZE[1]), device="cpu")
    print(model)
    torch.save(model, 'RegNet_model.pkl')

    if torch.cuda.is_available():
        model = nn.DataParallel(model)
        model = model.cuda()

    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=opt.lr, momentum=opt.momentum, weight_decay=opt.weight_decay, nesterov=True)
    best_acc1 = 0
    model.train()

    for epoch in range(opt.epochs):
        adjust_learning_rate(optimizer, epoch, opt.lr)
        train(training_generator, model, criterion, optimizer, epoch, writer)
        acc1 = validate(test_generator, model, criterion, epoch, writer)

        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        save_checkpoint({
            "epoch": epoch + 1,
            "state_dict": model.state_dict(),
            "best_acc1": best_acc1,
            "optimizer": optimizer.state_dict(),
        }, is_best, opt.saved_path)


def train(train_loader, model, criterion, optimizer, epoch, writer):
    batch_time = AverageMeter("Time", ":6.3f")
    data_time = AverageMeter("Data", ":6.3f")
    losses = AverageMeter("Loss", ":.4e")
    top1 = AverageMeter("Acc@1", ":6.2f")
    top5 = AverageMeter("Acc@5", ":6.2f")
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))
    model.train()
    end = time.time()
    num_iter_per_epoch = len(train_loader)
    for i, (images, target) in enumerate(train_loader):
        data_time.update(time.time() - end)
        if torch.cuda.is_available():
            images = images.cuda()
            target = target.cuda()

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.detach().item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        writer.add_scalar('Train/Loss', losses.avg, epoch * num_iter_per_epoch + i)
        writer.add_scalar('Train/Top1_acc', top1.avg, epoch * num_iter_per_epoch + i)
        writer.add_scalar('Train/Top5_acc', top5.avg, epoch * num_iter_per_epoch + i)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:
            progress.display(i)


def validate(val_loader, model, criterion, epoch, writer):
    batch_time = AverageMeter("Time", ":6.3f")
    losses = AverageMeter("Loss", ":.4e")
    top1 = AverageMeter("Acc@1", ":6.2f")
    top5 = AverageMeter("Acc@5", ":6.2f")
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix="Test: ")

    # switch to evaluate mode
    model.eval()
    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if torch.cuda.is_available():
                images = images.cuda()
                target = target.cuda()

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 10 == 0:
                progress.display(i)

        print(" * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}"
              .format(top1=top1, top5=top5))
        writer.add_scalar('Test/Loss', losses.avg, epoch)
        writer.add_scalar('Test/Top1_acc', top1.avg, epoch)
        writer.add_scalar('Test/Top5_acc', top5.avg, epoch)

    return top1.avg


def save_checkpoint(state, is_best, saved_path, filename="checkpoint.pth.tar"):
    file_path = os.path.join(saved_path, filename)
    torch.save(state, file_path)
    if is_best:
        shutil.copyfile(file_path, os.path.join(saved_path, "best_checkpoint.pth.tar"))


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=":f"):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = "{name} {val" + self.fmt + "} ({avg" + self.fmt + "})"
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print("\t".join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = "{:" + str(num_digits) + "d}"
        return "[" + fmt + "/" + fmt.format(num_batches) + "]"


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

if __name__ == "__main__":
    opt = get_args()
    start = datetime.datetime.now()
    main(opt)
    end = datetime.datetime.now()
    print(end-start)


# **Train a config model**

**Generate configs**

In [ ]:
import os
import sys
import argparse
import importlib
import configparser
from tqdm import tqdm

import torch
import autotorch as at

from thop import profile, clever_format
sys.path.append('../content/drive/My Drive')
# from RegNet_Research.RegNet_search import arch
from RegNet_Research.RegNet_search.arch import regnet

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def get_args():
    # data settings
    parser = argparse.ArgumentParser(description='RegNet-AutoTorch')
    # config files
    parser.add_argument('--arch', type=str, default='regnet',
                        help='network type (default: regnet)')
    parser.add_argument('--config-file', type=str, default='/content/drive/My Drive/RegNet_Research/RegNet_search/configs/RegNetX-0.4GF-ISR.ini',
                        help='target config file prefix')
    # input size
    parser.add_argument('--crop-size', type=int, default=[16, 256],
                        help='crop image size')
    # target flops
    parser.add_argument('--gflops', type=float, default=0.4,
                         help='expected flops')
    parser.add_argument('--eps', type=float, default=2e-2,
                         help='eps for expected flops')
    # num configs
    parser.add_argument('--num-configs', type=int, default=16,
                        help='num of expected configs')
    parser.add_argument('-f')
    parser = parser

    args = parser.parse_args()
    return args


def is_config_valid(arch, cfg, target_flops, input_tensor, eps):
    model = arch.config_network(cfg.dump_config())
    with HiddenPrints():
        flops, params = profile(model, inputs=[input_tensor, ])
    str_flops, str_params = clever_format([flops, params], "%.3f")
    # print('Flops', str_flops, '|','Target Flops', target_flops, '|', 'Parameters', str_params)
    return flops <= (1. + eps) * target_flops and \
flops >= (1. - eps) * target_flops


def main():
    args = get_args()
    input_tensor = torch.rand(1, 3, args.crop_size[0], args.crop_size[1])
    print(input_tensor.shape)
    arch = regnet
    cfg_generator = arch.GenConfg()
    searcher = at.searcher.RandomSearcher(cfg_generator.cs)

    valid = 0
    pbar = tqdm(range(args.num_configs))
    while valid < args.num_configs:
        config = searcher.get_config()
        cfg = cfg_generator.sample(**config)
        if is_config_valid(arch, cfg, args.gflops*1e8, input_tensor, args.eps):
            pbar.update()
            print(cfg)
            valid += 1
            cfg.dump_config(f'{args.config_file}-{valid}.ini')
            print('-------------------------------------------------------------')


if __name__ == '__main__':
    main()


  0%|          | 0/16 [00:00<?, ?it/s]

torch.Size([1, 3, 16, 256])


  6%|▋         | 1/16 [14:20<3:35:10, 860.68s/it]

{'bottleneck_ratio': 1, 'initial_width': 31, 'slope': 36.462893678088854, 'quantized_param': 2.292143842271364, 'network_depth': 15, 'group_width': 24}
-------------------------------------------------------------


 12%|█▎        | 2/16 [14:49<2:22:37, 611.27s/it]

...

{'bottleneck_ratio': 1, 'initial_width': 26, 'slope': 29.721009567855567, 'quantized_param': 2.383812021035117, 'network_depth': 17, 'group_width': 24}
-------------------------------------------------------------


 88%|████████▊ | 14/16 [3:33:39<24:33, 736.87s/it]

{'bottleneck_ratio': 1, 'initial_width': 25, 'slope': 53.5851858866527, 'quantized_param': 2.2345538284443016, 'network_depth': 15, 'group_width': 24}
-------------------------------------------------------------


 94%|█████████▍| 15/16 [3:58:11<15:57, 957.36s/it]

{'bottleneck_ratio': 1, 'initial_width': 19, 'slope': 34.79829828568338, 'quantized_param': 2.765354253727912, 'network_depth': 17, 'group_width': 16}
-------------------------------------------------------------


**Train generated model**

# **Train for ISRMyo-I**

In [ ]:
import h5py
import math
import random
import copy
import numpy as np
import torch
import torch.utils.data as Data
import torch.nn as nn
import torchvision.models as model
import matplotlib.pyplot as plt
import torch.nn.functional as F
import datetime
from torch.nn import Conv2d, Module, Linear, BatchNorm2d, ReLU
from torch.nn.modules.utils import _pair


class SplAtConv2d(Module):
    """Split-Attention Conv2d
    """
    def __init__(self, in_channels, channels, kernel_size, stride=(1, 1), padding=(0, 0),
                 dilation=(1, 1), groups=1, bias=True,
                 radix=2, reduction_factor=4,
                 rectify=False, rectify_avg=False, norm_layer=None,
                 dropblock_prob=0.0, **kwargs):
        super(SplAtConv2d, self).__init__()
        padding = _pair(padding)
        self.rectify = rectify and (padding[0] > 0 or padding[1] > 0)
        self.rectify_avg = rectify_avg
        inter_channels = max(in_channels*radix//reduction_factor, 32)
        self.radix = radix
        self.cardinality = groups
        self.channels = channels
        self.dropblock_prob = dropblock_prob
        self.conv = Conv2d(in_channels, channels*radix, kernel_size, stride, padding, dilation,
                               groups=groups*radix, bias=bias, **kwargs)
        self.use_bn = norm_layer is not None
        if self.use_bn:
            self.bn0 = norm_layer(channels*radix)
        self.relu = ReLU(inplace=True)
        self.fc1 = Conv2d(channels, inter_channels, 1, groups=self.cardinality)
        if self.use_bn:
            self.bn1 = norm_layer(inter_channels)
        self.fc2 = Conv2d(inter_channels, channels*radix, 1, groups=self.cardinality)
        if dropblock_prob > 0.0:
            self.dropblock = DropBlock2D(dropblock_prob, 3)
        self.rsoftmax = rSoftMax(radix, groups)

    def forward(self, x):
        x = self.conv(x)
        if self.use_bn:
            x = self.bn0(x)
        if self.dropblock_prob > 0.0:
            x = self.dropblock(x)
        x = self.relu(x)

        batch, rchannel = x.shape[:2]
        if self.radix > 1:
            if torch.__version__ < '1.5':
                splited = torch.split(x, int(rchannel//self.radix), dim=1)
            else:
                splited = torch.split(x, rchannel//self.radix, dim=1)
            gap = sum(splited)
        else:
            gap = x
        gap = F.adaptive_avg_pool2d(gap, 1)
        gap = self.fc1(gap)

        if self.use_bn:
            gap = self.bn1(gap)
        gap = self.relu(gap)

        atten = self.fc2(gap)
        atten = self.rsoftmax(atten).view(batch, -1, 1, 1)

        if self.radix > 1:
            if torch.__version__ < '1.5':
                attens = torch.split(atten, int(rchannel//self.radix), dim=1)
            else:
                attens = torch.split(atten, rchannel//self.radix, dim=1)
            out = sum([att*split for (att, split) in zip(attens, splited)])
        else:
            out = atten * x
        return out.contiguous()

class rSoftMax(nn.Module):
    def __init__(self, radix, cardinality):
        super().__init__()
        self.radix = radix
        self.cardinality = cardinality

    def forward(self, x):
        batch = x.size(0)
        if self.radix > 1:
            x = x.view(batch, self.cardinality, self.radix, -1).transpose(1, 2)
            x = F.softmax(x, dim=1)
            x = x.reshape(batch, -1)
        else:
            x = torch.sigmoid(x)
        return x



"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


# Assuming that we are on a CUDA machine, this should print a CUDA device:


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

gpu_cuda = torch.cuda.is_available()
print(gpu_cuda)

Database = h5py.File('/content/drive/My Drive/ISRMyo-I.h5', 'r')

train_set = Database['train_set']
test_set = Database['test_set']

# obtain training data & testing data
train_bangli_data = train_set['train_yajie']
test_bangli_data = test_set['test_yajie']

# extract label from group
train_bangli_label = train_bangli_data[:, 0].reshape(len(train_bangli_data), 1)
test_bangli_label = test_bangli_data[:, 0].reshape(len(test_bangli_data), 1)
print('train_label: ', train_bangli_label.shape)

# obtain items and labels from train_bangli_data
tl = torch.LongTensor(train_bangli_label)
sl = torch.LongTensor(test_bangli_label)

# training data pro-processing
train_bangli_data -= np.mean(train_bangli_data, axis=0)
train_bangli_data /= np.max(train_bangli_data)                # regularisation


# testing data pro-processing
test_bangli_data -= np.mean(test_bangli_data, axis=0)
test_bangli_data /= np.max(test_bangli_data)                  # regularisation


# obtain train data
train_data = []
for i in range(len(train_bangli_data)):
    td = train_bangli_data[i][1:].reshape(16, 256)
    train_data.append(td)
training_data = np.array(train_data)
training_data = torch.from_numpy(training_data)
training_data = torch.unsqueeze(training_data, dim=1).type(torch.FloatTensor)
training_data = torch.cat((training_data, training_data, training_data), 1)

# obtain test data
test_data = []
for i in range(len(test_bangli_data)):
    sd = test_bangli_data[i][1:].reshape(16, 256)
    # print(sd.shape)

    test_data.append(sd)
testing_data = np.array(test_data)
testing_data = torch.from_numpy(testing_data)
testing_data = torch.unsqueeze(testing_data, dim=1).type(torch.FloatTensor)
testing_data = torch.cat((testing_data, testing_data, testing_data), 1)

# CUDA training & testing
if gpu_cuda:
    training_data = training_data.cuda()
    testing_data = testing_data.cuda()

# transfer labels to tensor
training_label = torch.squeeze(tl)
testing_label = torch.squeeze(sl)
if gpu_cuda:
    training_label = training_label.cuda()
    testing_label = testing_label.cuda()

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


define training database


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

BATCH_SIZE = 128
print(training_data.shape, training_label.shape)
dataset = Data.TensorDataset(training_data, training_label)  # build dataset

Loader = Data.DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    shuffle=True, 
#     num_workers=8, 
    # drop_last=False
)

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


Architecture of ResNeSt


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
"""ResNet variants"""


class DropBlock2D(object):
    def __init__(self, *args, **kwargs):
        raise NotImplementedError


class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        """Global average pooling over the input's spatial dimensions"""
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, inputs):
        return nn.functional.adaptive_avg_pool2d(inputs, 1).view(inputs.size(0), -1)


class Bottleneck(nn.Module):
    """ResNet Bottleneck
    """
    # pylint: disable=unused-argument
    expansion = 4
    def __init__(self, inplanes, planes, stride=1, downsample=None,
                 radix=1, cardinality=1, bottleneck_width=64,
                 avd=False, avd_first=False, dilation=1, is_first=False,
                 rectified_conv=False, rectify_avg=False,
                 norm_layer=None, dropblock_prob=0.0, last_gamma=False):
        super(Bottleneck, self).__init__()
        group_width = int(planes * (bottleneck_width / 64.)) * cardinality
        self.conv1 = nn.Conv2d(inplanes, group_width, kernel_size=1, bias=False)
        self.bn1 = norm_layer(group_width)
        self.dropblock_prob = dropblock_prob
        self.radix = radix
        self.avd = avd and (stride > 1 or is_first)
        self.avd_first = avd_first

        if self.avd:
            self.avd_layer = nn.AvgPool2d(3, stride, padding=1)
            stride = 1

        if dropblock_prob > 0.0:
            self.dropblock1 = DropBlock2D(dropblock_prob, 3)
            if radix == 1:
                self.dropblock2 = DropBlock2D(dropblock_prob, 3)
            self.dropblock3 = DropBlock2D(dropblock_prob, 3)

        if radix >= 1:
            self.conv2 = SplAtConv2d(
                group_width, group_width, kernel_size=3,
                stride=stride, padding=dilation,
                dilation=dilation, groups=cardinality, bias=False,
                radix=radix, rectify=rectified_conv,
                rectify_avg=rectify_avg,
                norm_layer=norm_layer,
                dropblock_prob=dropblock_prob)
        else:
            self.conv2 = nn.Conv2d(
                group_width, group_width, kernel_size=3, stride=stride,
                padding=dilation, dilation=dilation,
                groups=cardinality, bias=False)
            self.bn2 = norm_layer(group_width)

        self.conv3 = nn.Conv2d(
            group_width, planes * 4, kernel_size=1, bias=False)
        self.bn3 = norm_layer(planes*4)

        if last_gamma:
            from torch.nn.init import zeros_
            zeros_(self.bn3.weight)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.dilation = dilation
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        if self.dropblock_prob > 0.0:
            out = self.dropblock1(out)
        out = self.relu(out)

        if self.avd and self.avd_first:
            out = self.avd_layer(out)

        out = self.conv2(out)
        if self.radix == 0:
            out = self.bn2(out)
            if self.dropblock_prob > 0.0:
                out = self.dropblock2(out)
            out = self.relu(out)

        if self.avd and not self.avd_first:
            out = self.avd_layer(out)

        out = self.conv3(out)
        out = self.bn3(out)
        if self.dropblock_prob > 0.0:
            out = self.dropblock3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, radix=1, groups=1, bottleneck_width=64,
                 num_classes=13, dilated=False, dilation=1,
                 deep_stem=False, stem_width=64, avg_down=False,
                 rectified_conv=False, rectify_avg=False,
                 avd=False, avd_first=False,
                 final_drop=0.0, dropblock_prob=0,
                 last_gamma=False, norm_layer=nn.BatchNorm2d):
        self.cardinality = groups
        self.bottleneck_width = bottleneck_width
        # ResNet-D params
        self.inplanes = stem_width*2 if deep_stem else 64
        self.avg_down = avg_down
        self.last_gamma = last_gamma
        # ResNeSt params
        self.radix = radix
        self.avd = avd
        self.avd_first = avd_first

        super(ResNet, self).__init__()
        self.rectified_conv = rectified_conv
        self.rectify_avg = rectify_avg
        conv_layer = nn.Conv2d
        conv_kwargs = {'average_mode': rectify_avg} if rectified_conv else {}
        if deep_stem:
            self.conv1 = nn.Sequential(
                conv_layer(3, stem_width, kernel_size=3, stride=2, padding=1, bias=False, **conv_kwargs),
                norm_layer(stem_width),
                nn.ReLU(inplace=True),
                conv_layer(stem_width, stem_width, kernel_size=3, stride=1, padding=1, bias=False, **conv_kwargs),
                norm_layer(stem_width),
                nn.ReLU(inplace=True),
                conv_layer(stem_width, stem_width*2, kernel_size=3, stride=1, padding=1, bias=False, **conv_kwargs),
            )
        else:
            self.conv1 = conv_layer(3, 64, kernel_size=7, stride=2, padding=3,
                                   bias=False, **conv_kwargs)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], norm_layer=norm_layer, is_first=False)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, norm_layer=norm_layer)
        if dilated or dilation == 4:
            self.layer3 = self._make_layer(block, 256, layers[2], stride=1,
                                           dilation=2, norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
            self.layer4 = self._make_layer(block, 512, layers[3], stride=1,
                                           dilation=4, norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
        elif dilation==2:
            self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                           dilation=1, norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
            self.layer4 = self._make_layer(block, 512, layers[3], stride=1,
                                           dilation=2, norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
        else:
            self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                           norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
            self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                           norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
        self.avgpool = GlobalAvgPool2d()
        self.drop = nn.Dropout(final_drop) if final_drop > 0.0 else None
        self.fc = nn.Sequential(
            nn.Linear(512 * block.expansion, num_classes)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, norm_layer):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1, norm_layer=None,
                    dropblock_prob=0.0, is_first=True):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            down_layers = []
            if self.avg_down:
                if dilation == 1:
                    down_layers.append(nn.AvgPool2d(kernel_size=stride, stride=stride,
                                                    ceil_mode=True, count_include_pad=False))
                else:
                    down_layers.append(nn.AvgPool2d(kernel_size=1, stride=1,
                                                    ceil_mode=True, count_include_pad=False))
                down_layers.append(nn.Conv2d(self.inplanes, planes * block.expansion,
                                             kernel_size=1, stride=1, bias=False))
            else:
                down_layers.append(nn.Conv2d(self.inplanes, planes * block.expansion,
                                             kernel_size=1, stride=stride, bias=False))
            down_layers.append(norm_layer(planes * block.expansion))
            downsample = nn.Sequential(*down_layers)

        layers = []
        if dilation == 1 or dilation == 2:
            layers.append(block(self.inplanes, planes, stride, downsample=downsample,
                                radix=self.radix, cardinality=self.cardinality,
                                bottleneck_width=self.bottleneck_width,
                                avd=self.avd, avd_first=self.avd_first,
                                dilation=1, is_first=is_first, rectified_conv=self.rectified_conv,
                                rectify_avg=self.rectify_avg,
                                norm_layer=norm_layer, dropblock_prob=dropblock_prob,
                                last_gamma=self.last_gamma))
        elif dilation == 4:
            layers.append(block(self.inplanes, planes, stride, downsample=downsample,
                                radix=self.radix, cardinality=self.cardinality,
                                bottleneck_width=self.bottleneck_width,
                                avd=self.avd, avd_first=self.avd_first,
                                dilation=2, is_first=is_first, rectified_conv=self.rectified_conv,
                                rectify_avg=self.rectify_avg,
                                norm_layer=norm_layer, dropblock_prob=dropblock_prob,
                                last_gamma=self.last_gamma))
        else:
            raise RuntimeError("=> unknown dilation size: {}".format(dilation))

        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes,
                                radix=self.radix, cardinality=self.cardinality,
                                bottleneck_width=self.bottleneck_width,
                                avd=self.avd, avd_first=self.avd_first,
                                dilation=dilation, rectified_conv=self.rectified_conv,
                                rectify_avg=self.rectify_avg,
                                norm_layer=norm_layer, dropblock_prob=dropblock_prob,
                                last_gamma=self.last_gamma))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        #x = x.view(x.size(0), -1)
        x = torch.flatten(x, 1)
        if self.drop:
            x = self.drop(x)
        x = self.fc(x)
        return x


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


Architecture of Basic CNN


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

class Basic_CNN(nn.Module):
    def __init__(self):
        super(Basic_CNN, self).__init__()
        self.conv1 = nn.Sequential(  
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
        )
        self.f_con_1 = nn.Sequential(nn.Linear(128 * 32 * 2, 128 * 32 * 2), nn.Sigmoid())
        self.out = torch.nn.Sequential(nn.Linear(128 * 32 * 2, 13))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.f_con_1(x)
        output = self.out(x)
        return output

# net = ResNet(Bottleneck, [3, 4, 6, 3], radix=2, groups=1, bottleneck_width=64, deep_stem=True, stem_width=32,
#              avg_down=True, avd=True, avd_first=False)
# net = torch.load('/content/RegNet_model_ISR.pkl')
net = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)
# net = torch.hub.load('pytorch/vision:v0.6.0', 'resnext50_32x4d', pretrained=False)
# net = Basic_CNN()


if gpu_cuda:
    net = net.cuda()


class Regularization(nn.Module):
    def __init__(self, model, weight_decay, p=2):
        super(Regularization, self).__init__()
        if weight_decay <= 0:
            print("param weight_decay can not <=0")
            exit(0)
        self.model = model
        self.weight_decay = weight_decay
        self.p = p
        self.weight_list = self.get_weight(model)
        # self.weight_info(self.weight_list)

    def forward(self, model):
        self.weight_list = self.get_weight(model)  
        reg_loss = self.regularization_loss(self.weight_list, self.weight_decay, p=self.p)
        return reg_loss

    def get_weight(self, model):
        '''
        :param model:
        :return:
        '''
        weight_list = []
        for name, param in model.named_parameters():
            if 'weight' in name:
                weight = (name, param)
                weight_list.append(weight)
        return weight_list

    def regularization_loss(self, weight_list, weight_decay, p=2):
        reg_loss = 0
        for name, w in weight_list:
            l2_reg = torch.norm(w, p=p)
            reg_loss = reg_loss + l2_reg

        reg_loss = weight_decay * reg_loss
        return reg_loss


weight_decay = 0.0001

if gpu_cuda:
    net = net.cuda()
    if weight_decay > 0:
        reg_loss = Regularization(net, weight_decay, p=2).cuda()
    else:
        print("no regularization")
else:
    if weight_decay > 0:
        reg_loss = Regularization(net, weight_decay, p=2)
    else:
        print("no regularization")

# optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()  # the target label is not one-hotted

acc_his = []

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


training start


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

TRAIN_REPETITION = 1
ACC_HIS = []
LOSS_HIS = []
LOSS_HIS_TEST = []
TRAIN_ACC_HIS = []
Final_acc_his = []
repetition = 1
start = datetime.datetime.now()
for _ in range(TRAIN_REPETITION):
    EPOCH = 100
    net_train = copy.deepcopy(net)
    acc_his = []
    train_acc_his = []
    for epoch in range(EPOCH):
        Loss_history = []
        if epoch > 50:
            LR = 0.001
        else:
            LR = 0.01
        optimizer = torch.optim.Adam(net_train.parameters(), lr=LR, betas=(0.5, 0.99))
        # optimizer = torch.optim.SGD(net_train.parameters(), lr=LR, momentum=0.9, weight_decay=0.0005)
        net_train.train()
        for i, (batch_x, batch_y) in enumerate(Loader):
            x = random.randint(0, 189)
            y = random.randint(0, 7)
            for j in range(63):
                for p in range(7):
                    batch_x[:, :, y + p, x + j] = 0
            output = net_train(batch_x)
            loss = loss_func(output, batch_y)
            if weight_decay > 0:
                loss += reg_loss(net_train)
            total_loss = loss
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            if gpu_cuda:
                Loss_history.append(loss.cuda().data.cpu().numpy())
                pred_x = torch.max(output, dim=1)[1].cuda().data.cpu().numpy()
                accu = float((pred_x == batch_y.cuda().data.cpu().numpy()).sum()) / float(batch_y.size(0))
            else:
                Loss_history.append(loss.data.numpy())
                pred_x = torch.max(output, dim=1)[1].data.numpy()
                accu = float((pred_x == batch_y.data.numpy()).sum()) / float(batch_y.size(0))

        ave_loss = np.mean(np.array(Loss_history))
        # cnn_test.load_state_dict(torch.load('ISRMyo-I_CNN_parameters.pkl'))
        with torch.no_grad():
            # net_train.eval()
            test_output = net_train(testing_data)
            test_loss = loss_func(test_output, testing_label)
            if weight_decay > 0:
                test_loss += reg_loss(net_train)
            total_test_loss = test_loss
            optimizer.zero_grad()
            if gpu_cuda:
                print("REPETITION is : ", repetition)
                print("Prediction is : ", np.argmax(test_output.data.cpu().numpy(), axis=1))
                print("Reality is :    ", testing_label.cuda().data.cpu().numpy(), testing_label.shape)

                accuracy = float((np.argmax(test_output.data.cpu().numpy(),
                                            axis=1) == testing_label.cuda().data.cpu().numpy()).sum()) / float(
                    testing_label.size(0))
                print('Epoch: ', epoch, '| train loss: %.4f' % ave_loss, '| test loss: %.4f' % total_test_loss,
                      '| train accuracy: %.3f' % accu, '| test accuracy: %.3f' % accuracy, '| LR: ', LR)
                print(
                    '======================================================================================================')
            else:
                print("REPETITION is : ", repetition)
                print("Prediction is : ", np.argmax(test_output.data.numpy(), axis=1))
                print("Reality is :    ", testing_label.data.numpy(), testing_label.shape)
                accuracy = float(
                    (np.argmax(test_output.data.numpy(), axis=1) == testing_label.data.numpy()).sum()) / float(
                    testing_label.size(0))
                print('Epoch: ', epoch, '| train loss: %.4f' % ave_loss, '| test loss: %.4f' % total_test_loss,
                      '| train accuracy: %.3f' % accu, '| test accuracy: %.3f' % accuracy, '| LR: ', LR)
                print(
                    '======================================================================================================')

        #             print("train", batch_x, testing_data)
        acc_his.append(accuracy)
        train_acc_his.append(accu)
        LOSS_HIS.append(ave_loss)

    ACC_HIS.append(acc_his)
    TRAIN_ACC_HIS.append(train_acc_his)
    repetition += 1
# torch.save(net.state_dict(), 'Ninapro_Multi-Stream_parameters.pkl')
end = datetime.datetime.now()
acc_his = np.array(acc_his)
train_acc_his = np.array(train_acc_his)
LOSS_HIS = np.array(LOSS_HIS)

np.savetxt("/content/drive/My Drive/Training_his_ISR_ResNet/Train_loss_his_s4.txt", LOSS_HIS)
np.savetxt('/content/drive/My Drive/Training_his_ISR_ResNet/Train_Acc_his_s4.txt', train_acc_his)
np.savetxt('/content/drive/My Drive/Training_his_ISR_ResNet/Test_Acc_his_s4.txt', acc_his)
a=np.loadtxt("/content/drive/My Drive/Training_his_ISR_ResNet/Test_Acc_his_s4.txt", delimiter=',')
print('FINAL TEST ACCURACY IS: ', np.max(acc_his))

print(end-start)
plt.title('Result Analysis')
plt.plot(LOSS_HIS, color='red', linestyle='-', label='Train Accuracy')
plt.legend(loc=0, prop={'size': 5})

plt.xlabel("EPOCH")
plt.ylabel("ACCURACY/LOSS")
plt.show()

# **Train a GAN Model**


In [ ]:
import h5py
import math
import random
import numpy as np
import torch
import torch.utils.data as Data
import torch.nn as nn
import copy
import argparse
# %matplotlib inline
# import torchsummary
import matplotlib.pyplot as plt
import torchvision.models as model
import torch.nn.functional as F
from torch.nn import Conv2d, Module, Linear, BatchNorm2d, ReLU
from torch.nn.modules.utils import _pair
from torch.nn import init
from sklearn.model_selection import train_test_split

# torch.manual_seed(1)
# np.random.seed(1)

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


# Assuming that we are on a CUDA machine, this should print a CUDA device:


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

gpu_cuda = torch.cuda.is_available()
print(gpu_cuda)
CUDA_LAUNCH_BLOCKING = 1

# Database = h5py.File('NINAPRO_1.h5', 'r')
Database = h5py.File('/content/drive/My Drive/NINAPRO_1.h5', 'r')

dataset1 = Database['E1']

train_set_E1 = dataset1['E1_train']
train_set_E1 = np.array(train_set_E1)
test_set_E1 = dataset1['E1_test']
test_set_E1 = np.array(test_set_E1)

train_E1_set = train_set_E1
test_E1_set = test_set_E1

print(train_E1_set.shape, test_E1_set.shape)

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


define training database


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

BATCH_SIZE = 128


def Data_transfer(train_E1_set, test_E1_set):
    # extract label from group
    list_dataset = []
    testing_label_list = []
    testing_data_list = []
    rep = 1
    for _ in range(rep):
        train_E1_label = []
        for i in range(len(train_E1_set)):
            train_E1_label.append(train_E1_set[i][0])
        train_E1_label = np.array(train_E1_label)
        train_E1_label = train_E1_label.reshape(len(train_E1_label), 1) - 1

        test_E1_label = []
        for i in range(len(test_E1_set)):
            test_E1_label.append(test_E1_set[i][0])
        test_E1_label = np.array(test_E1_label)
        test_E1_label = test_E1_label.reshape(len(test_E1_label), 1) - 1
        tl = torch.LongTensor(train_E1_label)
        sl = torch.LongTensor(test_E1_label)

        # obtain train data
        train_data = []
        for i in range(len(train_E1_set)):
            td = train_E1_set[i][1:].reshape(20, 10)
            td = np.rot90(td, 1)
            train_data.append(td)
        train_data = np.array(train_data)
        training_data = torch.from_numpy(train_data)
        training_data = torch.unsqueeze(training_data, dim=1).type(torch.FloatTensor)
        training_data = torch.cat((training_data, training_data, training_data), 1)

        # obtain test data
        test_data = []
        for i in range(len(test_E1_set)):
            sd = test_E1_set[i][1:].reshape(20, 10)
            sd = np.rot90(sd, 1)
            # print(sd.shape)
            test_data.append(sd)
        test_data = np.array(test_data)
        testing_data = torch.from_numpy(test_data)
        testing_data = torch.unsqueeze(testing_data, dim=1).type(torch.FloatTensor)
        testing_data = torch.cat((testing_data, testing_data, testing_data), 1)

        # CUDA training & testing
        if gpu_cuda:
            training_data = training_data.cuda()
            testing_data = testing_data.cuda()

        print('train_max : ', torch.max(training_data), 'test_max : ', torch.max(testing_data))
        # transfer labels to tensor

        training_label = torch.squeeze(tl)
        testing_label = torch.squeeze(sl)
        if gpu_cuda:
            training_label = training_label.cuda()
            testing_label = testing_label.cuda()

        dataset = Data.TensorDataset(training_data, training_label)  # build dataset
        # print(dataset)

        Loader = Data.DataLoader(
            dataset=dataset,
            batch_size=128,
            shuffle=True, 
            # num_workers=12, 
            drop_last=True
        )

        list_dataset.append(Loader)
        testing_data_list.append(testing_data)
        testing_label_list.append(testing_label)
        print(testing_data.shape)

    return list_dataset, testing_data_list, testing_label_list


list_dataset, testing_data_list, testing_label_list = Data_transfer(train_E1_set, test_E1_set)

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


Architecture of CNN


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
"""ResNet variants"""


class Basic_CNN(nn.Module):
    def __init__(self):
        super(Basic_CNN, self).__init__()
        self.unsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv1 = nn.Sequential(

            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
        )
        self.lc1 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
        )
        self.lc2 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
        )

        self.f_con = nn.Sequential(
            nn.Linear(256 * 20 * 10, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.8),
        )
        self.output = torch.nn.Sequential(
            nn.Linear(1024, 52 + 1),
        )
        self.validity = torch.nn.Sequential(
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.lc1(x)
        x = self.lc2(x)
        x = x.view(x.size(0), -1)
        x = self.f_con(x)
        classes = self.output(x)
        validity = self.validity(x)
        return validity, classes


net = Basic_CNN()

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print('Net_Parameters_Count : ', count_params(net))
# print(net)

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


training start


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


class Regularization(nn.Module):
    def __init__(self, model, weight_decay, p=2):
        super(Regularization, self).__init__()
        if weight_decay <= 0:
            print("param weight_decay can not <=0")
            exit(0)
        self.model = model
        self.weight_decay = weight_decay
        self.p = p
        self.weight_list = self.get_weight(model)
        # self.weight_info(self.weight_list)

    def forward(self, model):
        self.weight_list = self.get_weight(model) 
        reg_loss = self.regularization_loss(self.weight_list, self.weight_decay, p=self.p)
        return reg_loss

    def get_weight(self, model):
        weight_list = []
        for name, param in model.named_parameters():
            if 'weight' in name:
                weight = (name, param)
                weight_list.append(weight)
        return weight_list

    def regularization_loss(self, weight_list, weight_decay, p=2):
        reg_loss = 0
        for name, w in weight_list:
            l2_reg = torch.norm(w, p=p)
            reg_loss = reg_loss + l2_reg

        reg_loss = weight_decay * reg_loss
        return reg_loss


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


Architecture of Generator


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

NOISE_DIM = 52


def Generator(batch_size):
    a = np.random.randn(batch_size, NOISE_DIM)
    if gpu_cuda:
        fake_emg = torch.from_numpy(a).cuda()
    else:
        fake_emg = torch.from_numpy(a)

    return fake_emg


class Generator_Net(nn.Module):
    def __init__(self, noise_dim=NOISE_DIM):
        super(Generator_Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(noise_dim, 20 * 10 * 1024),
            nn.ReLU(),
            nn.BatchNorm1d(20 * 10 * 1024),
        )

        self.conv = nn.Sequential(
            nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.ConvTranspose2d(in_channels=256, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        # print('Generator size : ', x.shape)
        x = self.fc(x)
        x = x.view(x.shape[0], 1024, 20, 10)
        x = self.conv(x)
        # print('fake input shape : ', x.shape)
        return x


G_net = Generator_Net()
print('Generator_Parameters_Count : ', count_params(G_net))

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


loss function define

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

# optimize all cnn parameters
if gpu_cuda:
    auxiliary_loss = nn.CrossEntropyLoss().cuda()  # the target label is not one-hotted
    adversarial_loss = nn.BCELoss().cuda()
else:
    auxiliary_loss = nn.CrossEntropyLoss()  # the target label is not one-hotted
    adversarial_loss = nn.BCELoss()


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


# Initialize generator and discriminator
# net.apply(weights_init_normal)
# G_net.apply(weights_init_normal)

if gpu_cuda:
    net = net.cuda()
    G_net = G_net.cuda()
    weight_decay = 0.0001
    if weight_decay > 0:
        reg_loss = Regularization(net, weight_decay, p=2).cuda()
    else:
        print("no regularization")
else:
    weight_decay = 0.0001
    if weight_decay > 0:
        reg_loss = Regularization(net, weight_decay, p=2)
    else:
        print("no regularization")

FloatTensor = torch.cuda.FloatTensor if gpu_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if gpu_cuda else torch.LongTensor

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


training start


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

TRAIN_REPETITION = 1
ACC_HIS = []
Final_acc_his = []
repetition = 1
# LR = 0.0003
n_critic = 5
clip_value = 0.01
num_classes = 52

for _ in range(TRAIN_REPETITION):
    EPOCH = 100
    Net_D = copy.deepcopy(net)
    Net_G = copy.deepcopy(G_net)
    test_acc_his = []
    train_acc_his = []
    judge_acc_his = []
    Loader = list_dataset[_]
    testing_data = testing_data_list[_]
    testing_label = testing_label_list[_]
    for epoch in range(EPOCH):
        G_Loss_history = []
        D_Loss_history = []
        if 60 >= epoch > 30:
            LR = 0.005
        elif epoch > 60:
            LR = 0.001
        else:
            LR = 0.01
        optimizer_D = torch.optim.Adam(Net_D.parameters(), lr=LR, betas=(0.5, 0.99))
        optimizer_G = torch.optim.Adam(Net_G.parameters(), lr=LR, betas=(0.5, 0.99))
        # optimizer_G = torch.optim.RMSprop(Net_G.parameters())
        Net_D.train()
        Net_G.train()
        for i, (imgs, labels) in enumerate(Loader):
            batch_size = len(imgs)
            # Adversarial ground truths
            valid = FloatTensor(batch_size, 1).fill_(1.0)
            fake = FloatTensor(batch_size, 1).fill_(0.0)
            fake_aux_gt = LongTensor(batch_size).fill_(num_classes)  # fake_multi-class-label

            # ---------------------
            #  Train Generator
            # ---------------------

            optimizer_G.zero_grad()

            # Generate a batch of images
            z = Generator(batch_size).float()
            fake_imgs = Net_G(z)

            # Loss measures generator's ability to fool the discriminator
            validity, _ = Net_D(fake_imgs)
            # print('validity : ', validity.shape, 'fake_label : ', _.shape)     # 512, 1 //  512, 53
            g_loss = adversarial_loss(validity, valid)
            g_loss.backward()
            optimizer_G.step()
            # print(imgs.shape, fake_imgs.shape)

            # --------------------
            #  Train Discriminator
            # --------------------

            optimizer_D.zero_grad()

            # Loss for real images
            real_pred, real_aux = Net_D(imgs)
            d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

            # Loss for fake images
            fake_pred, fake_aux = Net_D(fake_imgs.detach())
            d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, fake_aux_gt)) / 2

            # Total discriminator loss
            d_loss = (d_real_loss + d_fake_loss) / 2

            if weight_decay > 0:
                d_loss += reg_loss(Net_D)

            if gpu_cuda:
                pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
                gt = np.concatenate([labels.data.cpu().numpy(), fake_aux_gt.data.cpu().numpy()], axis=0)
            else:
                pred = np.concatenate([real_aux.data.numpy(), fake_aux.data.numpy()], axis=0)
                # print(pred.shape)
                gt = np.concatenate([labels.data.numpy(), fake_aux_gt.data.numpy()], axis=0)
            d_acc = np.mean(np.argmax(pred, axis=1) == gt)

            d_loss.backward()
            optimizer_D.step()

            if gpu_cuda:
                G_Loss_history.append(g_loss.cuda().data.cpu().numpy())
                D_Loss_history.append(d_loss.cuda().data.cpu().numpy())
                accu = float(
                    (np.argmax(real_aux.data.cpu().numpy(), axis=1) == labels.cuda().data.cpu().numpy()).sum()) / float(
                    labels.size(0))
            else:
                G_Loss_history.append(g_loss.data.numpy())
                D_Loss_history.append(d_loss.data.numpy())
                accu = float((np.argmax(real_aux, axis=1).data.numpy() == labels.data.numpy()).sum()) / float(
                    labels.size(0))

            d_ave_loss = np.mean(np.array(D_Loss_history))
            g_ave_loss = np.mean(np.array(G_Loss_history))

        with torch.no_grad():
            Net_D.eval()
            Net_G.eval()
            _, test_output = Net_D(testing_data)
            if gpu_cuda:
                accuracy = float((np.argmax(test_output.data.cpu().numpy(),
                                            axis=1) == testing_label.cuda().data.cpu().numpy()).sum()) / float(
                    testing_label.size(0))
                print("REPETITION is : ", repetition)
                print("Prediction is : ", np.argmax(test_output.data.cpu().numpy(), axis=1))
                print("Reality is :    ", testing_label.cuda().data.cpu().numpy(), testing_label.shape)
                print('Epoch: ', epoch, '| train loss: %.4f' % d_ave_loss, '| generator loss: %.4f' % g_ave_loss,
                      '| train accuracy: %.3f' % accu, '| test accuracy: %.3f' % accuracy,
                      '| Recog accuracy: %.3f' % d_acc,
                      '| LR: ', LR)
                print(
                    '======================================================================================================')
            else:
                accuracy = float(
                    (np.argmax(test_output.data.numpy(), axis=1) == testing_label.data.numpy()).sum()) / float(
                    testing_label.size(0))
                print("REPETITION is : ", repetition)
                print("Prediction is : ", np.argmax(test_output.data.numpy(), axis=1))
                print("Reality is :    ", testing_label.data.numpy(), testing_label.shape)
                print('Epoch: ', epoch, '| train loss: %.4f' % d_ave_loss, '| generator loss: %.4f' % g_ave_loss,
                      '| train accuracy: %.3f' % accu, '| test accuracy: %.3f' % accuracy,
                      '| Recog accuracy: %.3f' % d_acc,
                      '| LR: ', LR)
                print(
                    '======================================================================================================')

        #             print("train", batch_x, testing_data)
        test_acc_his.append(accuracy)
        train_acc_his.append(accu)
        judge_acc_his.append(d_acc)

    ACC_HIS.append(test_acc_his)
    repetition += 1
torch.save(Net_D.state_dict(), 'Ninapro_SGAN_DIS_parameters.pkl')
torch.save(Net_G.state_dict(), 'Ninapro_SGAN_GEN_parameters.pkl')

# plt.style.use(['science', 'grid', 'ieee', 'no-latex'])
plt.title('Result Analysis')
plt.plot(test_acc_his, color='green', linestyle='-', label='Test_Accuracy')
plt.plot(train_acc_his, color='red', linestyle='-', label='Train_Accuracy')
plt.plot(judge_acc_his, color='skyblue', linestyle='-', label='Discriminate_Accuracy')
plt.legend(loc=0, prop={'size': 5})

plt.xlabel("EPOCH")
plt.ylabel("ACCURACY")
plt.show()



# **Train Basic CNN & RegNetX Model for NINAPRO**

In [ ]:
import h5py
import math
import random
import copy
import numpy as np
import torch
import torch.utils.data as Data
import torch.nn as nn
# %matplotlib inline
import datetime
import torchvision.models as model
import matplotlib.pyplot as plt
from scipy import signal
import torch.nn.functional as F
from torch.nn import Conv2d, Module, Linear, BatchNorm2d, ReLU
from torch.nn.modules.utils import _pair


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


# Assuming that we are on a CUDA machine, this should print a CUDA device:


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


gpu_cuda = torch.cuda.is_available()
print(gpu_cuda)
CUDA_LAUNCH_BLOCKING = 1

Database = h5py.File('/content/drive/My Drive/NINAPRO_1.h5', 'r')

dataset1 = Database['E27']

train_set_E1 = dataset1['E27_train']
train_set_E1 = np.array(train_set_E1)
test_set_E1 = dataset1['E27_test']
test_set_E1 = np.array(test_set_E1)

train_E1_set = train_set_E1
test_E1_set = test_set_E1

print(train_E1_set.shape, test_E1_set.shape)

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


define training database


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
def Data_transfer(train_E1_set, test_E1_set):
    # extract label from group
    list_dataset = []
    testing_label_list = []
    testing_data_list = []
    rep = 1
    for _ in range(rep):
        train_E1_label = []
        for i in range(len(train_E1_set)):
            train_E1_label.append(train_E1_set[i][0])
        train_E1_label = np.array(train_E1_label)
        train_E1_label = train_E1_label.reshape(len(train_E1_label), 1) - 1

        test_E1_label = []
        for i in range(len(test_E1_set)):
            test_E1_label.append(test_E1_set[i][0])
        test_E1_label = np.array(test_E1_label)
        test_E1_label = test_E1_label.reshape(len(test_E1_label), 1) - 1
        tl = torch.LongTensor(train_E1_label)
        sl = torch.LongTensor(test_E1_label)

        # obtain train data
        train_data = []
        for i in range(len(train_E1_set)):
            td = train_E1_set[i][1:].reshape(20, 10)
            td = np.rot90(td, 1)
            train_data.append(td)
        train_data = np.array(train_data)
        training_data = torch.from_numpy(train_data)
        training_data = torch.unsqueeze(training_data, dim=1).type(torch.FloatTensor)
        training_data = torch.cat((training_data, training_data, training_data), 1)

        # obtain test data
        test_data = []
        for i in range(len(test_E1_set)):
            sd = test_E1_set[i][1:].reshape(20, 10)
            sd = np.rot90(sd, 1)
            test_data.append(sd)
        test_data = np.array(test_data)
        testing_data = torch.from_numpy(test_data)
        testing_data = torch.unsqueeze(testing_data, dim=1).type(torch.FloatTensor)
        testing_data = torch.cat((testing_data, testing_data, testing_data), 1)

        # CUDA training & testing
        if gpu_cuda:
            training_data = training_data.cuda()
            testing_data = testing_data.cuda()

        print('train_max : ', torch.max(training_data), 'test_max : ', torch.max(testing_data))
        # transfer labels to tensor

        training_label = torch.squeeze(tl)
        testing_label = torch.squeeze(sl)
        if gpu_cuda:
            training_label = training_label.cuda()
            testing_label = testing_label.cuda()

        dataset = Data.TensorDataset(training_data, training_label)  # build dataset
        # print(dataset)

        Loader = Data.DataLoader(
            dataset=dataset,
            batch_size=128,
            shuffle=True,  
            # num_workers=12,  
            drop_last=True
        )

        list_dataset.append(Loader)
        testing_data_list.append(testing_data)
        testing_label_list.append(testing_label)
        print(testing_data.shape)

    return list_dataset, testing_data_list, testing_label_list


list_dataset, testing_data_list, testing_label_list = Data_transfer(train_E1_set, test_E1_set)

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


Architecture of ResNeSt


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
"""ResNet variants"""

class SplAtConv2d(Module):
    """Split-Attention Conv2d
    """
    def __init__(self, in_channels, channels, kernel_size, stride=(1, 1), padding=(0, 0),
                 dilation=(1, 1), groups=1, bias=True,
                 radix=2, reduction_factor=4,
                 rectify=False, rectify_avg=False, norm_layer=None,
                 dropblock_prob=0.0, **kwargs):
        super(SplAtConv2d, self).__init__()
        padding = _pair(padding)
        self.rectify = rectify and (padding[0] > 0 or padding[1] > 0)
        self.rectify_avg = rectify_avg
        inter_channels = max(in_channels*radix//reduction_factor, 32)
        self.radix = radix
        self.cardinality = groups
        self.channels = channels
        self.dropblock_prob = dropblock_prob
        self.conv = Conv2d(in_channels, channels*radix, kernel_size, stride, padding, dilation,
                               groups=groups*radix, bias=bias, **kwargs)
        self.use_bn = norm_layer is not None
        if self.use_bn:
            self.bn0 = norm_layer(channels*radix)
        self.relu = ReLU(inplace=True)
        self.fc1 = Conv2d(channels, inter_channels, 1, groups=self.cardinality)
        if self.use_bn:
            self.bn1 = norm_layer(inter_channels)
        self.fc2 = Conv2d(inter_channels, channels*radix, 1, groups=self.cardinality)
        if dropblock_prob > 0.0:
            self.dropblock = DropBlock2D(dropblock_prob, 3)
        self.rsoftmax = rSoftMax(radix, groups)

    def forward(self, x):
        x = self.conv(x)
        if self.use_bn:
            x = self.bn0(x)
        if self.dropblock_prob > 0.0:
            x = self.dropblock(x)
        x = self.relu(x)

        batch, rchannel = x.shape[:2]
        if self.radix > 1:
            if torch.__version__ < '1.5':
                splited = torch.split(x, int(rchannel//self.radix), dim=1)
            else:
                splited = torch.split(x, rchannel//self.radix, dim=1)
            gap = sum(splited)
        else:
            gap = x
        gap = F.adaptive_avg_pool2d(gap, 1)
        gap = self.fc1(gap)

        if self.use_bn:
            gap = self.bn1(gap)
        gap = self.relu(gap)

        atten = self.fc2(gap)
        atten = self.rsoftmax(atten).view(batch, -1, 1, 1)

        if self.radix > 1:
            if torch.__version__ < '1.5':
                attens = torch.split(atten, int(rchannel//self.radix), dim=1)
            else:
                attens = torch.split(atten, rchannel//self.radix, dim=1)
            out = sum([att*split for (att, split) in zip(attens, splited)])
        else:
            out = atten * x
        return out.contiguous()

class rSoftMax(nn.Module):
    def __init__(self, radix, cardinality):
        super().__init__()
        self.radix = radix
        self.cardinality = cardinality

    def forward(self, x):
        batch = x.size(0)
        if self.radix > 1:
            x = x.view(batch, self.cardinality, self.radix, -1).transpose(1, 2)
            x = F.softmax(x, dim=1)
            x = x.reshape(batch, -1)
        else:
            x = torch.sigmoid(x)
        return x

class DropBlock2D(object):
    def __init__(self, *args, **kwargs):
        raise NotImplementedError


class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        """Global average pooling over the input's spatial dimensions"""
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, inputs):
        return nn.functional.adaptive_avg_pool2d(inputs, 1).view(inputs.size(0), -1)


class Bottleneck(nn.Module):
    """ResNet Bottleneck
    """
    # pylint: disable=unused-argument
    expansion = 4
    def __init__(self, inplanes, planes, stride=1, downsample=None,
                 radix=1, cardinality=1, bottleneck_width=64,
                 avd=False, avd_first=False, dilation=1, is_first=False,
                 rectified_conv=False, rectify_avg=False,
                 norm_layer=None, dropblock_prob=0.0, last_gamma=False):
        super(Bottleneck, self).__init__()
        group_width = int(planes * (bottleneck_width / 64.)) * cardinality
        self.conv1 = nn.Conv2d(inplanes, group_width, kernel_size=1, bias=False)
        self.bn1 = norm_layer(group_width)
        self.dropblock_prob = dropblock_prob
        self.radix = radix
        self.avd = avd and (stride > 1 or is_first)
        self.avd_first = avd_first

        if self.avd:
            self.avd_layer = nn.AvgPool2d(3, stride, padding=1)
            stride = 1

        if dropblock_prob > 0.0:
            self.dropblock1 = DropBlock2D(dropblock_prob, 3)
            if radix == 1:
                self.dropblock2 = DropBlock2D(dropblock_prob, 3)
            self.dropblock3 = DropBlock2D(dropblock_prob, 3)

        if radix >= 1:
            self.conv2 = SplAtConv2d(
                group_width, group_width, kernel_size=3,
                stride=stride, padding=dilation,
                dilation=dilation, groups=cardinality, bias=False,
                radix=radix, rectify=rectified_conv,
                rectify_avg=rectify_avg,
                norm_layer=norm_layer,
                dropblock_prob=dropblock_prob)
        else:
            self.conv2 = nn.Conv2d(
                group_width, group_width, kernel_size=3, stride=stride,
                padding=dilation, dilation=dilation,
                groups=cardinality, bias=False)
            self.bn2 = norm_layer(group_width)

        self.conv3 = nn.Conv2d(
            group_width, planes * 4, kernel_size=1, bias=False)
        self.bn3 = norm_layer(planes*4)

        if last_gamma:
            from torch.nn.init import zeros_
            zeros_(self.bn3.weight)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.dilation = dilation
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        if self.dropblock_prob > 0.0:
            out = self.dropblock1(out)
        out = self.relu(out)

        if self.avd and self.avd_first:
            out = self.avd_layer(out)

        out = self.conv2(out)
        if self.radix == 0:
            out = self.bn2(out)
            if self.dropblock_prob > 0.0:
                out = self.dropblock2(out)
            out = self.relu(out)

        if self.avd and not self.avd_first:
            out = self.avd_layer(out)

        out = self.conv3(out)
        out = self.bn3(out)
        if self.dropblock_prob > 0.0:
            out = self.dropblock3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, radix=1, groups=1, bottleneck_width=64,
                 num_classes=52, dilated=False, dilation=1,
                 deep_stem=False, stem_width=64, avg_down=False,
                 rectified_conv=False, rectify_avg=False,
                 avd=False, avd_first=False,
                 final_drop=0.0, dropblock_prob=0,
                 last_gamma=False, norm_layer=nn.BatchNorm2d):
        self.cardinality = groups
        self.bottleneck_width = bottleneck_width
        # ResNet-D params
        self.inplanes = stem_width*2 if deep_stem else 64
        self.avg_down = avg_down
        self.last_gamma = last_gamma
        # ResNeSt params
        self.radix = radix
        self.avd = avd
        self.avd_first = avd_first

        super(ResNet, self).__init__()
        self.rectified_conv = rectified_conv
        self.rectify_avg = rectify_avg
        conv_layer = nn.Conv2d
        conv_kwargs = {'average_mode': rectify_avg} if rectified_conv else {}
        if deep_stem:
            self.conv1 = nn.Sequential(
                conv_layer(3, stem_width, kernel_size=3, stride=2, padding=1, bias=False, **conv_kwargs),
                norm_layer(stem_width),
                nn.ReLU(inplace=True),
                conv_layer(stem_width, stem_width, kernel_size=3, stride=1, padding=1, bias=False, **conv_kwargs),
                norm_layer(stem_width),
                nn.ReLU(inplace=True),
                conv_layer(stem_width, stem_width*2, kernel_size=3, stride=1, padding=1, bias=False, **conv_kwargs),
            )
        else:
            self.conv1 = conv_layer(3, 64, kernel_size=7, stride=2, padding=3,
                                   bias=False, **conv_kwargs)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 32, layers[0], norm_layer=norm_layer, is_first=False)
        self.layer2 = self._make_layer(block, 64, layers[1], stride=2, norm_layer=norm_layer)
        if dilated or dilation == 4:
            self.layer3 = self._make_layer(block, 128, layers[2], stride=1,
                                           dilation=2, norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
            self.layer4 = self._make_layer(block, 256, layers[3], stride=1,
                                           dilation=4, norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
        elif dilation==2:
            self.layer3 = self._make_layer(block, 128, layers[2], stride=2,
                                           dilation=1, norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
            self.layer4 = self._make_layer(block, 256, layers[3], stride=1,
                                           dilation=2, norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
        else:
            self.layer3 = self._make_layer(block, 128, layers[2], stride=1,
                                           norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
            self.layer4 = self._make_layer(block, 256, layers[3], stride=1,
                                           norm_layer=norm_layer,
                                           dropblock_prob=dropblock_prob)
        self.avgpool = GlobalAvgPool2d()
        self.drop = nn.Dropout(final_drop) if final_drop > 0.0 else None
        self.fc = nn.Sequential(
            nn.Linear(256 * 2 * 3 * block.expansion, num_classes)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, norm_layer):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1, norm_layer=None,
                    dropblock_prob=0.0, is_first=True):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            down_layers = []
            if self.avg_down:
                if dilation == 1:
                    down_layers.append(nn.AvgPool2d(kernel_size=stride, stride=stride,
                                                    ceil_mode=True, count_include_pad=False))
                else:
                    down_layers.append(nn.AvgPool2d(kernel_size=1, stride=1,
                                                    ceil_mode=True, count_include_pad=False))
                down_layers.append(nn.Conv2d(self.inplanes, planes * block.expansion,
                                             kernel_size=1, stride=1, bias=False))
            else:
                down_layers.append(nn.Conv2d(self.inplanes, planes * block.expansion,
                                             kernel_size=1, stride=stride, bias=False))
            down_layers.append(norm_layer(planes * block.expansion))
            downsample = nn.Sequential(*down_layers)

        layers = []
        if dilation == 1 or dilation == 2:
            layers.append(block(self.inplanes, planes, stride, downsample=downsample,
                                radix=self.radix, cardinality=self.cardinality,
                                bottleneck_width=self.bottleneck_width,
                                avd=self.avd, avd_first=self.avd_first,
                                dilation=1, is_first=is_first, rectified_conv=self.rectified_conv,
                                rectify_avg=self.rectify_avg,
                                norm_layer=norm_layer, dropblock_prob=dropblock_prob,
                                last_gamma=self.last_gamma))
        elif dilation == 4:
            layers.append(block(self.inplanes, planes, stride, downsample=downsample,
                                radix=self.radix, cardinality=self.cardinality,
                                bottleneck_width=self.bottleneck_width,
                                avd=self.avd, avd_first=self.avd_first,
                                dilation=2, is_first=is_first, rectified_conv=self.rectified_conv,
                                rectify_avg=self.rectify_avg,
                                norm_layer=norm_layer, dropblock_prob=dropblock_prob,
                                last_gamma=self.last_gamma))
        else:
            raise RuntimeError("=> unknown dilation size: {}".format(dilation))

        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes,
                                radix=self.radix, cardinality=self.cardinality,
                                bottleneck_width=self.bottleneck_width,
                                avd=self.avd, avd_first=self.avd_first,
                                dilation=dilation, rectified_conv=self.rectified_conv,
                                rectify_avg=self.rectify_avg,
                                norm_layer=norm_layer, dropblock_prob=dropblock_prob,
                                last_gamma=self.last_gamma))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = torch.flatten(x, 1)
        if self.drop:
            x = self.drop(x)
        x = self.fc(x)
        return x

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


Architecture of CNN


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
"""ResNet variants"""


class Basic_CNN(nn.Module):
    def __init__(self):
        super(Basic_CNN, self).__init__()
        self.unsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv1 = nn.Sequential(

            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
        )
        self.lc1 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
        )
        self.lc2 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
        )

        self.f_con = nn.Sequential(
            nn.Linear(256 * 20 * 10, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.8),
        )
        self.output = torch.nn.Sequential(
            nn.Linear(1024, 52),
        )


    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.lc1(x)
        x = self.lc2(x)
        x = x.view(x.size(0), -1)
        x = self.f_con(x)
        x = self.output(x)

        return x


net = Basic_CNN()
# net = ResNet(Bottleneck, [3, 4, 6, 3], radix=2, groups=2, bottleneck_width=64, 
#              deep_stem=True, stem_width=32, avg_down=True, avd=True, avd_first=False)
# net = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)
# net = torch.load('/content/RegNet_model.pkl')


# print(net)
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print('Net_Parameters_Count : ', count_params(net))


class Regularization(nn.Module):
    def __init__(self, model, weight_decay, p=2):
        '''
        :param model 模型
        :param weight_decay:正则化参数
        :param p: 范数计算中的幂指数值，默认求2范数,
                  当p=0为L2正则化,p=1为L1正则化
        '''
        super(Regularization, self).__init__()
        if weight_decay <= 0:
            print("param weight_decay can not <=0")
            exit(0)
        self.model = model
        self.weight_decay = weight_decay
        self.p = p
        self.weight_list = self.get_weight(model)
        # self.weight_info(self.weight_list)

    def forward(self, model):
        self.weight_list = self.get_weight(model)  # obtain updated weights
        reg_loss = self.regularization_loss(self.weight_list, self.weight_decay, p=self.p)
        return reg_loss

    def get_weight(self, model):
        weight_list = []
        for name, param in model.named_parameters():
            if 'weight' in name:
                weight = (name, param)
                weight_list.append(weight)
        return weight_list

    def regularization_loss(self, weight_list, weight_decay, p=2):
        reg_loss = 0
        for name, w in weight_list:
            l2_reg = torch.norm(w, p=p)
            reg_loss = reg_loss + l2_reg

        reg_loss = weight_decay * reg_loss
        return reg_loss


weight_decay = 0.0001

if gpu_cuda:
    net = net.cuda()
    if weight_decay > 0:
        reg_loss = Regularization(net, weight_decay, p=2).cuda()
    else:
        print("no regularization")
else:
    if weight_decay > 0:
        reg_loss = Regularization(net, weight_decay, p=2)
    else:
        print("no regularization")

# optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()  # the target label is not one-hotted

acc_his = []

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


training start


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

TRAIN_REPETITION = 1
ACC_HIS = []
LOSS_HIS = []
LOSS_HIS_TEST = []
TRAIN_ACC_HIS = []
Final_acc_his = []
repetition = 1
start = datetime.datetime.now()
for _ in range(TRAIN_REPETITION):
    EPOCH = 60
    net_train = copy.deepcopy(net)
    acc_his = []
    train_acc_his = []
    Loader = list_dataset[_]
    testing_data = testing_data_list[_]
    testing_label = testing_label_list[_]
    for epoch in range(EPOCH):
        Loss_history = []
        if 40 >= epoch > 20:
            LR = 0.005
        elif 60 >= epoch > 40:
            LR = 0.001
        else:
            LR = 0.01
        optimizer = torch.optim.Adam(net_train.parameters(), lr=LR, betas=(0.5, 0.99))
        # optimizer = torch.optim.SGD(net_train.parameters(), lr=LR, momentum=0.9, weight_decay=0.0005)
        net_train.train()
        for i, (batch_x, batch_y) in enumerate(Loader):
            output = net_train(batch_x)
            loss = loss_func(output, batch_y)
            if weight_decay > 0:
                loss += reg_loss(net_train)
            total_loss = loss
            optimizer.zero_grad()
            total_loss.backward()
            # loss.backward()
            optimizer.step()
            if gpu_cuda:
                Loss_history.append(loss.cuda().data.cpu().numpy())
                pred_x = torch.max(output, dim=1)[1].cuda().data.cpu().numpy()
                accu = float((pred_x == batch_y.cuda().data.cpu().numpy()).sum()) / float(batch_y.size(0))
            else:
                Loss_history.append(loss.data.numpy())
                pred_x = torch.max(output, dim=1)[1].data.numpy()
                accu = float((pred_x == batch_y.data.numpy()).sum()) / float(batch_y.size(0))

        ave_loss = np.mean(np.array(Loss_history))
        # cnn_test.load_state_dict(torch.load('ISRMyo-I_CNN_parameters.pkl'))

        with torch.no_grad():
            net_train.eval()
            test_output = net_train(testing_data)
            test_loss = loss_func(test_output, testing_label)
            # if weight_decay > 0:
            #     test_loss += reg_loss(net_train)
            total_test_loss = test_loss
            optimizer.zero_grad()
            # g = make_dot(test_output)
            # g.view()
            if gpu_cuda:
                print("REPETITION is : ", repetition)
                print("Prediction is : ", np.argmax(test_output.data.cpu().numpy(), axis=1))
                print("Reality is :    ", testing_label.cuda().data.cpu().numpy(), testing_label.shape)

                accuracy = float((np.argmax(test_output.data.cpu().numpy(),
                                            axis=1) == testing_label.cuda().data.cpu().numpy()).sum()) / float(
                    testing_label.size(0))
                print('Epoch: ', epoch, '| train loss: %.4f' % ave_loss, '| test loss: %.4f' % total_test_loss,
                      '| train accuracy: %.3f' % accu, '| test accuracy: %.3f' % accuracy, '| LR: ', LR)
                print(
                    '======================================================================================================')
            else:
                print("REPETITION is : ", repetition)
                print("Prediction is : ", np.argmax(test_output.data.numpy(), axis=1))
                print("Reality is :    ", testing_label.data.numpy(), testing_label.shape)
                accuracy = float(
                    (np.argmax(test_output.data.numpy(), axis=1) == testing_label.data.numpy()).sum()) / float(
                    testing_label.size(0))
                print('Epoch: ', epoch, '| train loss: %.4f' % ave_loss, '| test loss: %.4f' % total_test_loss,
                      '| train accuracy: %.3f' % accu, '| test accuracy: %.3f' % accuracy, '| LR: ', LR)
                print(
                    '======================================================================================================')

        #             print("train", batch_x, testing_data)
        acc_his.append(accuracy)
        train_acc_his.append(accu)
        LOSS_HIS.append(ave_loss)

    ACC_HIS.append(acc_his)
    TRAIN_ACC_HIS.append(train_acc_his)
    repetition += 1
# torch.save(net.state_dict(), 'Ninapro_Multi-Stream_parameters.pkl')
end = datetime.datetime.now()
acc_his = np.array(acc_his)
train_acc_his = np.array(train_acc_his)
LOSS_HIS = np.array(LOSS_HIS)

np.savetxt("/content/drive/My Drive/Training_his_NINAPRO_Basic/Train_Loss_His_NINAPRO1_S6.txt", LOSS_HIS)
np.savetxt('/content/drive/My Drive/Training_his_NINAPRO_Basic/Train_Accuracy_His_NINAPRO1_S6.txt', train_acc_his)
np.savetxt('/content/drive/My Drive/Training_his_NINAPRO_Basic/Test_Accuracy_His_NINAPRO1_S27.txt', acc_his)

a=np.loadtxt("/content/drive/My Drive/Training_his_NINAPRO_Basic/Train_Loss_His_NINAPRO1_S27.txt", delimiter=',')
print(type(a))
print('FINAL TEST ACCURACY IS: ', np.max(acc_his))
print(end-start)

# plt.style.use(['science', 'grid', 'ieee', 'no-latex'])
plt.title('Result Analysis')
plt.plot(LOSS_HIS, color='red', linestyle='-', label='Train Accuracy')
plt.legend(loc=0, prop={'size': 5})

plt.xlabel("EPOCH")
plt.ylabel("ACCURACY/LOSS")
plt.show()

## **Train a ResNet model**

In [ ]:
import h5py
import math
import random
import copy
import numpy as np
import torch
import torch.utils.data as Data
import torch.nn as nn
# %matplotlib inline
import datetime
import torchvision.models as model
import matplotlib.pyplot as plt
from scipy import signal

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


# Assuming that we are on a CUDA machine, this should print a CUDA device:


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


gpu_cuda = torch.cuda.is_available()
print(gpu_cuda)
CUDA_LAUNCH_BLOCKING = 1

# Database = h5py.File('NINAPRO_1.h5', 'r')
Database = h5py.File('/content/drive/My Drive/NINAPRO_1.h5', 'r')

dataset1 = Database['E1']

train_set_E1 = dataset1['E1_train']
train_set_E1 = np.array(train_set_E1)
test_set_E1 = dataset1['E1_test']
test_set_E1 = np.array(test_set_E1)

train_E1_set = train_set_E1
test_E1_set = test_set_E1

print(train_E1_set.shape, test_E1_set.shape)

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


define training database


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

BATCH_SIZE = 512


def Data_transfer(train_E1_set, test_E1_set):
    # extract label from group
    list_dataset = []
    testing_label_list = []
    testing_data_list = []
    rep = 1
    for _ in range(rep):
        # train_E1_set, test_E1_set = train_test_split(train_set, test_size=0.3)
        # print(train_E1_set.shape, test_E1_set.shape)

        train_E1_label = []
        for i in range(len(train_E1_set)):
            train_E1_label.append(train_E1_set[i][0])
        train_E1_label = np.array(train_E1_label)
        train_E1_label = train_E1_label.reshape(len(train_E1_label), 1) - 1
#         train_E1_label = np.vstack((train_E1_label, train_E1_label))
        # train_bangli_label = np.vstack((train_bangli_label, train_bangli_label))

        test_E1_label = []
        for i in range(len(test_E1_set)):
            test_E1_label.append(test_E1_set[i][0])
        test_E1_label = np.array(test_E1_label)
        test_E1_label = test_E1_label.reshape(len(test_E1_label), 1) - 1
        tl = torch.LongTensor(train_E1_label)
        sl = torch.LongTensor(test_E1_label)

        # obtain train data
        train_data = []
        for i in range(len(train_E1_set)):
            # print(train_E1_set[i][1:])
            td = train_E1_set[i][1:].reshape(20, 10)
            td = np.rot90(td, 1)
            # print('td: ', td.shape)
            train_data.append(td)
        train_data = np.array(train_data)
        training_data = torch.from_numpy(train_data)
        training_data = torch.unsqueeze(training_data, dim=1).type(torch.FloatTensor)
        training_data = torch.cat((training_data, training_data, training_data), 1)

        # obtain test data
        test_data = []
        for i in range(len(test_E1_set)):
            sd = test_E1_set[i][1:].reshape(20, 10)
            sd = np.rot90(sd, 1)
            # print(sd.shape)
            test_data.append(sd)
        test_data = np.array(test_data)
        testing_data = torch.from_numpy(test_data)
        testing_data = torch.unsqueeze(testing_data, dim=1).type(torch.FloatTensor)
        testing_data = torch.cat((testing_data, testing_data, testing_data), 1)
        # print('test data: ', testing_data, testing_data.shape, type(testing_data))


        # CUDA training & testing
        if gpu_cuda:
            training_data = training_data.cuda()
            testing_data = testing_data.cuda()

        print('train_max : ', torch.max(training_data), 'test_max : ', torch.max(testing_data))
        # transfer labels to tensor

        training_label = torch.squeeze(tl)
        testing_label = torch.squeeze(sl)
        if gpu_cuda:
            training_label = training_label.cuda()
            testing_label = testing_label.cuda()

        dataset = Data.TensorDataset(training_data, training_label)  # build dataset
        # print(dataset)

        Loader = Data.DataLoader(
            dataset=dataset,
            batch_size=512,
            shuffle=True,  # 是否采用随机训练 true：采用  false：不采用
            # num_workers=12,  # 进程数，提取次数，step循环次数，一般等于 数据个数/batch_size
            # drop_last=True
        )
        # Loader2 = Data.DataLoader(
        #     dataset=dataset,
        #     batch_size=128,
        #     shuffle=True,  # 是否采用随机训练 true：采用  false：不采用
        #     # num_workers=12,  # 进程数，提取次数，step循环次数，一般等于 数据个数/batch_size
        #     drop_last=True
        # )

        list_dataset.append(Loader)
        testing_data_list.append(testing_data)
        testing_label_list.append(testing_label)
        print(testing_data.shape)

    return list_dataset, testing_data_list, testing_label_list


list_dataset, testing_data_list, testing_label_list = Data_transfer(train_E1_set, test_E1_set)


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


Architecture of CNN


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
"""ResNet variants"""




class Basic_CNN(nn.Module):
    def __init__(self):
        super(Basic_CNN, self).__init__()
        self.unsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv1 = nn.Sequential(

            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
        )
        self.lc1 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            # nn.Dropout(0.5),
        )
        self.lc2 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
        )

        self.f_con = nn.Sequential(
            nn.Linear(256 * 20 * 10, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.8),
        )
        self.output = torch.nn.Sequential(
            nn.Linear(1024, 52),
        )


    def forward(self, x):
        # print(len(x))
        # print(x)
        # x = torch.reshape(x, (len(x), 1, 10, 20))
        # print(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.lc1(x)
        x = self.lc2(x)
        x = x.view(x.size(0), -1)
        x = self.f_con(x)
        x = self.output(x)

        return x

net = torch.hub.load('pytorch/vision:v0.6.0', 'resnext50_32x4d', pretrained=False)
# net = Basic_CNN()
# net = torch.load('/content/RegNet_model.pkl')


# print(net)
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print('Net_Parameters_Count : ', count_params(net))


class Regularization(nn.Module):
    def __init__(self, model, weight_decay, p=2):
        '''
        :param model 模型
        :param weight_decay:正则化参数
        :param p: 范数计算中的幂指数值，默认求2范数,
                  当p=0为L2正则化,p=1为L1正则化
        '''
        super(Regularization, self).__init__()
        if weight_decay <= 0:
            print("param weight_decay can not <=0")
            exit(0)
        self.model = model
        self.weight_decay = weight_decay
        self.p = p
        self.weight_list = self.get_weight(model)
        # self.weight_info(self.weight_list)

    def forward(self, model):
        self.weight_list = self.get_weight(model)  # 获得最新的权重
        reg_loss = self.regularization_loss(self.weight_list, self.weight_decay, p=self.p)
        return reg_loss

    def get_weight(self, model):
        '''
        获得模型的权重列表
        :param model:
        :return:
        '''
        weight_list = []
        for name, param in model.named_parameters():
            if 'weight' in name:
                weight = (name, param)
                weight_list.append(weight)
        return weight_list

    def regularization_loss(self, weight_list, weight_decay, p=2):
        '''
        计算张量范数
        :param weight_list:
        :param p: 范数计算中的幂指数值，默认求2范数
        :param weight_decay:
        :return:
        '''

        reg_loss = 0
        for name, w in weight_list:
            l2_reg = torch.norm(w, p=p)
            reg_loss = reg_loss + l2_reg

        reg_loss = weight_decay * reg_loss
        return reg_loss


weight_decay = 0.0001

if gpu_cuda:
    net = net.cuda()
    if weight_decay > 0:
        reg_loss = Regularization(net, weight_decay, p=2).cuda()
    else:
        print("no regularization")
else:
    if weight_decay > 0:
        reg_loss = Regularization(net, weight_decay, p=2)
    else:
        print("no regularization")

# optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()  # the target label is not one-hotted

acc_his = []

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


training start


"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

TRAIN_REPETITION = 1
ACC_HIS = []
LOSS_HIS = []
LOSS_HIS_TEST = []
TRAIN_ACC_HIS = []
Final_acc_his = []
repetition = 1
start = datetime.datetime.now()
for _ in range(TRAIN_REPETITION):
    EPOCH = 80
    net_train = copy.deepcopy(net)
    acc_his = []
    train_acc_his = []
    Loader = list_dataset[_]
    testing_data = testing_data_list[_]
    testing_label = testing_label_list[_]
    for epoch in range(EPOCH):
        Loss_history = []
        if 50 >= epoch > 30:
            LR = 0.005
        elif epoch > 50:
            LR = 0.001
        # elif epoch > 50:
        #     LR = 0.0003
        else:
            LR = 0.01
        # optimizer = torch.optim.Adam(net_train.parameters(), lr=LR, betas=(0.5, 0.99))
        optimizer = torch.optim.SGD(net_train.parameters(), lr=LR, momentum=0.9, weight_decay=0.0005)
        net_train.train()
        for i, (batch_x, batch_y) in enumerate(Loader):
            output = net_train(batch_x)
            loss = loss_func(output, batch_y)
            # if weight_decay > 0:
            #     loss += reg_loss(net_train)
            # total_loss = loss
            optimizer.zero_grad()
            # total_loss.backward()
            # loss.backward()
            optimizer.step()
            if gpu_cuda:
                Loss_history.append(loss.cuda().data.cpu().numpy())
                pred_x = torch.max(output, dim=1)[1].cuda().data.cpu().numpy()
                accu = float((pred_x == batch_y.cuda().data.cpu().numpy()).sum()) / float(batch_y.size(0))
            else:
                Loss_history.append(loss.data.numpy())
                pred_x = torch.max(output, dim=1)[1].data.numpy()
                accu = float((pred_x == batch_y.data.numpy()).sum()) / float(batch_y.size(0))

        ave_loss = np.mean(np.array(Loss_history))
        # cnn_test.load_state_dict(torch.load('ISRMyo-I_CNN_parameters.pkl'))

        with torch.no_grad():
            net_train.eval()
            test_output = net_train(testing_data)
            test_loss = loss_func(test_output, testing_label)
            if weight_decay > 0:
                test_loss += reg_loss(net_train)
            total_test_loss = test_loss
            optimizer.zero_grad()
            # g = make_dot(test_output)
            # g.view()
            if gpu_cuda:
                print("REPETITION is : ", repetition)
                print("Prediction is : ", np.argmax(test_output.data.cpu().numpy(), axis=1))
                print("Reality is :    ", testing_label.cuda().data.cpu().numpy(), testing_label.shape)

                accuracy = float((np.argmax(test_output.data.cpu().numpy(),
                                            axis=1) == testing_label.cuda().data.cpu().numpy()).sum()) / float(
                    testing_label.size(0))
                print('Epoch: ', epoch, '| train loss: %.4f' % ave_loss, '| test loss: %.4f' % total_test_loss,
                      '| train accuracy: %.3f' % accu, '| test accuracy: %.3f' % accuracy, '| LR: ', LR)
                print(
                    '======================================================================================================')
            else:
                print("REPETITION is : ", repetition)
                print("Prediction is : ", np.argmax(test_output.data.numpy(), axis=1))
                print("Reality is :    ", testing_label.data.numpy(), testing_label.shape)
                accuracy = float(
                    (np.argmax(test_output.data.numpy(), axis=1) == testing_label.data.numpy()).sum()) / float(
                    testing_label.size(0))
                print('Epoch: ', epoch, '| train loss: %.4f' % ave_loss, '| test loss: %.4f' % total_test_loss,
                      '| train accuracy: %.3f' % accu, '| test accuracy: %.3f' % accuracy, '| LR: ', LR)
                print(
                    '======================================================================================================')

        #             print("train", batch_x, testing_data)
        acc_his.append(accuracy)
        train_acc_his.append(accu)
        LOSS_HIS.append(ave_loss)

    ACC_HIS.append(acc_his)
    TRAIN_ACC_HIS.append(train_acc_his)
    repetition += 1
# torch.save(net.state_dict(), 'Ninapro_Multi-Stream_parameters.pkl')
end = datetime.datetime.now()
acc_his = np.array(acc_his)
train_acc_his = np.array(train_acc_his)
LOSS_HIS = np.array(LOSS_HIS)

# np.savetxt("/content/drive/My Drive/Training_his/Train_Loss_His_NINAPRO1_S1.txt", LOSS_HIS)
# np.savetxt('/content/drive/My Drive/Training_his/Train_Accuracy_His_NINAPRO1_S1.txt', train_acc_his)
# np.savetxt('/content/drive/My Drive/Training_his/Test_Accuracy_His_NINAPRO1_S1.txt', acc_his)

# a=np.loadtxt("/content/drive/My Drive/Training_his/Train_Loss_His_NINAPRO1_S1.txt", delimiter=',')
# print(type(a))
print('FINAL TEST ACCURACY IS: ', np.max(acc_his))
print(end-start)
# a = ACC_HIS[0]
# b = ACC_HIS[1]
# c = ACC_HIS[2]
# d = ACC_HIS[3]
# e = ACC_HIS[4]
# print("Final accuracy list: ", Final_acc_his)

# plt.style.use(['science', 'grid', 'ieee', 'no-latex'])
plt.title('Result Analysis')
# plt.plot(acc_his, color='green', linestyle='-', label='Test Accuracy')
plt.plot(LOSS_HIS, color='red', linestyle='-', label='Train Accuracy')
# plt.plot(c, color='skyblue', linestyle='-', label='Repetition 3')
# plt.plot(d, color='blue', linestyle='-', label='Repetition 4')
# plt.plot(e, color='yellow', linestyle='-', label='Repetition 5')
plt.legend(loc=0, prop={'size': 5})

plt.xlabel("EPOCH")
plt.ylabel("ACCURACY/LOSS")
plt.show()

True
(31610, 201) (13480, 201)
train_max :  tensor(4.4885, device='cuda:0') test_max :  tensor(4.4948, device='cuda:0')
torch.Size([13480, 3, 10, 20])


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


Net_Parameters_Count :  25028904
REPETITION is :  1
Prediction is :  [ 0  0  0 ... 47 47 47]
Reality is :     [ 0  0  0 ... 51 51 51] torch.Size([13480])
Epoch:  0 | train loss: 2.9785 | test loss: 2.1088 | train accuracy: 0.550 | test accuracy: 0.455 | LR:  0.01
REPETITION is :  1
Prediction is :  [ 0  0  0 ... 32 32 32]
Reality is :     [ 0  0  0 ... 51 51 51] torch.Size([13480])
Epoch:  1 | train loss: 1.0982 | test loss: 1.7361 | train accuracy: 0.730 | test accuracy: 0.592 | LR:  0.01
REPETITION is :  1
Prediction is :  [ 0  0  0 ... 32 32 51]
Reality is :     [ 0  0  0 ... 51 51 51] torch.Size([13480])
Epoch:  2 | train loss: 0.7107 | test loss: 1.4631 | train accuracy: 0.807 | test accuracy: 0.668 | LR:  0.01
REPETITION is :  1
Prediction is :  [ 0  0  0 ... 32 32 32]
Reality is :     [ 0  0  0 ... 51 51 51] torch.Size([13480])
Epoch:  3 | train loss: 0.4218 | test loss: 1.4680 | train accuracy: 0.860 | test accuracy: 0.688 | LR:  0.01
REPETITION is :  1
Prediction is :  [ 0  0 

KeyboardInterrupt: ignored